# Athena Autocall smart contract

## What is an Athena Autocall ?

### Input parameters :

* **European Down and In Put (EDIP)** : Capital Protection Barrier at Maturity. At maturity, 100% of capital is paid back if the underlying level is >= barrier, otherwise, (1-Performance) is paid back. Ex 70%.
* **Conditional coupon barrier**= Autocall barrier : If at an observation date, the underlying is lower than this barrier, nothing happens, if the underlying is equal or greater to this level, a coupon is paid and the capital is paid back (autocall event) (no risk exposition anymore). Ex 100%.
* **Coupon level** : The level of coupon distributed if the UL level > counditional coupon barrier * initial level. Ex 10% p.a.
* **Observation dates** : a calendar of observation dates when the level of UL is checked by the contract, and a coupon is paid / the product autocalled. Ex monthly, quarterly, semi-annually, annually.
* **Maturity** : Ex 1y.
* **Underlying** : Underlying asset of the product, its value will determine the outcome of each event depending on its value compared to its initial value. The volatility and other parameters of the UL will have determined the coupon level, risk protection barrier, ...
* **Price** : 100% - Mark Up -> The value that will be reimbursed at the end. The initial swap will be 100% of nominal but 95% will be reimbursed at the end.
* **Currency**: It is different from the currency of the UL, the UL currency does not matter since is is only used in levels %. The product can be traded in EUR, USD, ...
* **counterparty**: couterparty :/




### Setup the environement and import libs

In [ ]:
!pip install fredapi
import pandas as pd
import hashlib
import datetime
from datetime import datetime as dt

### Import the underlying of our financial asset

To import the underlying we use API from FRED

In [ ]:
from fredapi import Fred

def get_FRED_series(ticker):
    FRED_API_KEY = "cc8c5ce8b5f37ae0255d927b30d50982"
    fred = Fred(api_key=FRED_API_KEY)
    data = fred.get_series(ticker)
    data = data.dropna()
    data = pd.DataFrame(data)
    data.index = pd.to_datetime(data.index)
    return data

ETH = get_FRED_series("CBETHUSD")

ETH

,0
2016-05-18,13.18
2016-05-19,14.90
2016-05-20,14.17
2016-05-23,13.61
2016-05-24,12.77
...,...
2023-10-20,1605.03
2023-10-21,1629.55
2023-10-22,1663.57
2023-10-23,1766.77


### Build the class for a Autocall Athena

In [ ]:
class Autocall_Athena:

    def __init__(self, name, seller):
        self.balance = {}
        self.balance[self.hash_name(seller)] = 0  # Use the hash of the seller's name
        self.name = name
        self.total = 0
        self.last_mint_date = {}
        self.autocall_ids = []  # List to store unique autocall IDs and their ownership
        self.autocall_owners = {}  # Dictionary to map autocall IDs to their owners
        self.seller = seller

    def mint(self, client, autocall_id, coupon_autocall_barrier, valo_calendar, nominal, annual_coupon, protection_barrier, underlying, frequency):

        if autocall_id in self.autocall_ids:
            raise ValueError("Autocall ID already exists")

        self.seller = self.hash_name(self.seller)
        self.client = self.hash_name(client)  # Hash the account's name
        self.coupon_autocall_barrier = coupon_autocall_barrier
        self.valo_calendar = valo_calendar
        self.nominal = nominal
        self.annual_coupon = annual_coupon
        self.protection_barrier = protection_barrier
        self.underlying = self.get_FRED_series(underlying)
        self.frequency = frequency
        self.initial_date = self.get_current_date()
        self.finished = False
        self.autocall_id = autocall_id

        # Mint the tokens and update the balances
        if self.client not in self.balance:
            self.balance[self.client] = self.nominal
        else:
            self.balance[self.client] += (-self.nominal)
            self.balance[self.seller] += (self.nominal)
            print("the nominal : ", str(self.nominal), "has been transferd from client to seller")
        self.total += self.nominal

        # Check if there's a last mint date for the account
        last_mint = self.last_mint_date.get(self.client, None)

        # Update the last mint date for the account
        self.last_mint_date[self.client] = self.get_current_date()

        self.total += 1
        # Set the owners of the NFT according to id of the token
        self.autocall_ids.append(autocall_id)
        self.autocall_owners[autocall_id] = self.client
        return self.balance[self.client]

    def hash_name(self, name):
        # Compute the hash of the name using SHA-256
        return hashlib.sha256(name.encode()).hexdigest()

    def get_current_date(self):
        # Get the current date
        current_date = datetime.datetime.now().date()
        return current_date

    def get_FRED_series(self, underlying):
        FRED_API_KEY = "cc8c5ce8b5f37ae0255d927b30d50982"
        fred = Fred(api_key=FRED_API_KEY)
        data = fred.get_series(underlying)
        data = data.dropna()
        data = pd.DataFrame(data)
        data.index = pd.to_datetime(data.index)
        return data

    def balance_of(self, owner_address):
        """
        Get the balance of tokens owned by a specific address.

        Args:
        -----
            owner_address (str): The address of the owner.

        Returns:
        --------
            int: The number of tokens owned by the address.
        """
        print(self.balance)
        hash_owner = self.hash_name(owner_address)
        for autocall_id in self.autocall_ids:
            if self.autocall_owners[autocall_id] == hash_owner:
              print("The balance of " + owner_address + " is as follow:" )
              return self.balance[hash_owner]

    def owner_of(self, autocall_id):
        """
        Get the owner address of a specific token.

        Args:
        -----
            autocall_id (int): The unique identifier of the token.

        Returns:
        --------
            str: The address of the token's owner.
        """
        if autocall_id not in self.autocall_id:
            raise ValueError("Token ID does not exist")

        return self.autocall_owners[autocall_id]

    def safeTransferFrom(self, address_from, client, autocall_id):
        """
        Safely transfer a token from one address to another.

        Args:
            address_from (str): The address of the current owner of the token.
            client (str): The address to which the token will be transferred.
            autocall_id (int): The unique identifier of the token.
        """
        if self.owner_of(autocall_id) != address_from:
            raise ValueError("You cannot transfered because you are not the owner of " + self.autocall_owners[autocall_id])

        elif client == address_from:
            # Check if the sender and the receiver have the same adress
            print("You're currently the owner of " + self.autocall_owners[autocall_id] + " you can't transfered the token to yourself")

        else:
            # Transfer the token to the new owner
            self.autocall_owners[autocall_id] = client
            print(address_from + " has succesfully transfered to " + self.autocall_owners[autocall_id])
            print("The new owner is " + self.autocall_owners[autocall_id])

    def event_coupon_autocall(self):

        if self.get_current_date() in self.valo_calendar[:-1] :
          if self.underlying.loc[self.get_current_date(),0] >= self.coupon_autocall_barrier * self.underlying.loc[self.initial_date,0]:
            if self.finished==False:
              self.balance[self.client] += (self.nominal*self.annual_coupon/self.frequency + self.nominal)
              self.balance[self.seller] += (-self.nominal*self.annual_coupon/self.frequency - self.nominal)
              print("a coupon and the nominal : ", str((self.nominal*self.annual_coupon/self.frequency + self.nominal)), "has been transferd from seller to client. Product terminated.")
            else : self.balance[self.client] += 0

        elif self.get_current_date() == self.valo_calendar[-1]:
          if self.finished==False:

            if self.underlying.loc[self.get_current_date(),0] > self.coupon_autocall_barrier * self.underlying.loc[self.initial_date,0]:
              self.balance[self.client]  += (self.nominal*self.annual_coupon/self.frequency + self.nominal)
              self.balance[self.seller] += (-self.nominal*self.annual_coupon/self.frequency - self.nominal)
              print("a coupon and the nominal : ", str((self.nominal*self.annual_coupon/self.frequency + self.nominal)), "has been transferd from seller to client. Product terminated.")

            elif (self.underlying.loc[self.get_current_date(),0] <= self.coupon_autocall_barrier * self.underlying.loc[self.initial_date,0]) & (self.underlying.loc[self.get_current_date(),0] >= self.protection_barrier*self.underlying.loc[self.get_current_date(),0]):
              self.balance[self.client]  += self.nominal
              self.balance[self.seller] += (-self.nominal)
              print("the nominal : ", str(self.nominal), "has been reimbursed from seller to client. Product terminated")

            else:
              self.balance[self.client]  += self.nominal * ((self.underlying.loc[self.get_current_date(),0]/self.underlying.loc[self.initial_date,0]) -1)
              self.balance[self.seller] += (-self.nominal * ((self.underlying.loc[self.get_current_date(),0]/self.underlying.loc[self.initial_date,0]) -1))
              print("the remaining capital (- loss) : ", str(self.nominal * ((self.underlying.loc[self.get_current_date(),0]/self.underlying.loc[self.initial_date,0]) -1)), "has been reimbursed from seller to client. Product terminated")

          else : self.balance[self.client]  += 0

In [ ]:
autocall1 = Autocall_Athena(name="AC_test_1", seller='CACIB')

autocall1.mint(client="Kathy", autocall_id="ISIN1", coupon_autocall_barrier=1, valo_calendar=["2022-01-03", "2022-04-01", "2022-07-01", "2022-10-03"], nominal=1000, annual_coupon=0.1, protection_barrier=0.7, underlying='CBETHUSD', frequency=4)


#def mint(self, client, autocall_id, coupon_autocall_barrier, valo_calendar, nominal, annual_coupon, protection_barrier, underlying, frequency):


#mint(self, client, autocall_id, coupon_autocall_barrier, valo_calendar, nominal, annual_coupon, protection_barrier, underlying, frequency):

1000

In [ ]:
print("seller :", autocall1.seller)
print("")
print("client :",autocall1.client)
print("")
print("owners :",autocall1.autocall_owners)
print("")
print("autocall ID :", autocall1.autocall_id)
print("")
print(autocall1.balance_of("Kathy"))
print("")
print("autocall owner of ISIN 1 :", autocall1.owner_of("ISIN1"))

seller : 28c4d25bada78027ac04f9d3dc041e29099bcb558d8254ae0b9584bceefc3093

client : 2bc88a2ecf09d1fea7c54323cd9390e147feaf94dd54ff46373cea7a531d984e

owners : {'ISIN1': '2bc88a2ecf09d1fea7c54323cd9390e147feaf94dd54ff46373cea7a531d984e'}

autocall ID : ISIN1

{'28c4d25bada78027ac04f9d3dc041e29099bcb558d8254ae0b9584bceefc3093': 0, '2bc88a2ecf09d1fea7c54323cd9390e147feaf94dd54ff46373cea7a531d984e': 1000}
The balance of Kathy is as follow:
1000

autocall owner of ISIN 1 : 2bc88a2ecf09d1fea7c54323cd9390e147feaf94dd54ff46373cea7a531d984e


As an example, we will modify the class's get_current_date function to simulate transfers.

In [ ]:
class Autocall_Athena_Sim:

    def __init__(self, name, seller, curr_date):
        self.balance = {}
        self.balance[self.hash_name(seller)] = 0  # Use the hash of the seller's name
        self.name = name
        self.total = 0
        self.last_mint_date = {}
        self.autocall_ids = []  # List to store unique autocall IDs and their ownership
        self.autocall_owners = {}  # Dictionary to map autocall IDs to their owners
        self.seller = seller
        self.curr_date = curr_date

    def mint(self, client, autocall_id, coupon_autocall_barrier, valo_calendar, nominal, annual_coupon, protection_barrier, underlying, frequency):

        if autocall_id in self.autocall_ids:
            raise ValueError("Autocall ID already exists")

        self.seller = self.hash_name(self.seller)
        self.client = self.hash_name(client)  # Hash the account's name
        self.coupon_autocall_barrier = coupon_autocall_barrier
        self.valo_calendar = [dt.strptime(item, "%Y-%m-%d") for item in valo_calendar]
        self.nominal = nominal
        self.annual_coupon = annual_coupon
        self.protection_barrier = protection_barrier
        self.underlying = self.get_FRED_series(underlying)
        self.frequency = frequency
        self.initial_date = self.get_current_date()
        self.finished = False

        # Mint the tokens and update the balances
        if self.client not in self.balance:
            self.balance[self.client] = self.nominal
        else:
            self.balance[self.client] += (-self.nominal)
            self.balance[self.seller] += (self.nominal)
            print("the nominal : ", str(self.nominal), "has been transferd from client to seller")
        self.total += self.nominal

        # Check if there's a last mint date for the account
        last_mint = self.last_mint_date.get(self.client, None)

        # Update the last mint date for the account
        self.last_mint_date[self.client] = self.get_current_date()

        self.total += 1
        # Set the owners of the NFT according to id of the token
        self.autocall_ids.append(autocall_id)
        self.autocall_owners[autocall_id] = self.client
        return self.balance[self.client]

    def hash_name(self, name):
        # Compute the hash of the name using SHA-256
        return hashlib.sha256(name.encode()).hexdigest()

    def get_current_date(self):
        # Get the current date
        current_date = datetime.datetime.now().date()
        return current_date

    def get_FRED_series(self, underlying):
        FRED_API_KEY = "cc8c5ce8b5f37ae0255d927b30d50982"
        fred = Fred(api_key=FRED_API_KEY)
        data = fred.get_series(underlying)
        data = data.dropna()
        data = pd.DataFrame(data)
        data.index = pd.to_datetime(data.index)
        return data

    def balance_of(self, owner_address):
        """
        Get the balance of tokens owned by a specific address.

        Args:
        -----
            owner_address (str): The address of the owner.

        Returns:
        --------
            int: The number of tokens owned by the address.
        """
        print(self.balance)
        hash_owner = self.hash_name(owner_address)
        for autocall_id in self.autocall_ids:
            if self.autocall_owners[autocall_id] == hash_owner:
              print("The balance of " + owner_address + " is as follow:" )
              return self.balance[hash_owner]

    def owner_of(self, autocall_id):
        """
        Get the owner address of a specific token.

        Args:
        -----
            autocall_id (int): The unique identifier of the token.

        Returns:
        --------
            str: The address of the token's owner.
        """
        if autocall_id not in self.autocall_id:
            raise ValueError("Token ID does not exist")

        return self.autocall_owners[autocall_id]

    def safeTransferFrom(self, address_from, client, autocall_id):
        """
        Safely transfer a token from one address to another.

        Args:
            address_from (str): The address of the current owner of the token.
            client (str): The address to which the token will be transferred.
            autocall_id (int): The unique identifier of the token.
        """
        if self.owner_of(autocall_id) != address_from:
            raise ValueError("You cannot transfered because you are not the owner of " + self.autocall_owners[autocall_id])

        elif client == address_from:
            # Check if the sender and the receiver have the same adress
            print("You're currently the owner of " + self.autocall_owners[autocall_id] + " you can't transfered the token to yourself")

        else:
            # Transfer the token to the new owner
            self.autocall_owners[autocall_id] = client
            print(address_from + " has succesfully transfered to " + self.autocall_owners[autocall_id])
            print("The new owner is " + self.autocall_owners[autocall_id])

    def event_coupon_autocall(self):

        if self.curr_date in self.valo_calendar[:-1] :
          if self.underlying.loc[self.curr_date,0] >= self.coupon_autocall_barrier * self.underlying.loc[self.initial_date,0]:
            if self.finished==False:
              self.balance[self.client] += (self.nominal*self.annual_coupon/self.frequency + self.nominal)
              self.balance[self.seller] += (-self.nominal*self.annual_coupon/self.frequency - self.nominal)
              print("a coupon and the nominal : ", str((self.nominal*self.annual_coupon/self.frequency + self.nominal)), "has been transferd from seller to client. Product terminated.")
              self.finished=True
          else : self.balance[self.client] += 0

        elif self.curr_date == self.valo_calendar[-1] :
          if self.finished==False:

            if self.underlying.loc[self.curr_date,0] > self.coupon_autocall_barrier * self.underlying.loc[self.initial_date,0]:
              self.balance[self.client]  += (self.nominal*self.annual_coupon/self.frequency + self.nominal)
              self.balance[self.seller] += (-self.nominal*self.annual_coupon/self.frequency - self.nominal)
              self.finished=True
              print("a coupon and the nominal : ", str((self.annual_coupon/self.frequency + self.nominal)), "has been transferd from seller to client. Product terminated.")

            elif (self.underlying.loc[self.curr_date,0] <= self.coupon_autocall_barrier * self.underlying.loc[self.initial_date,0]) & (self.underlying.loc[self.curr_date,0] >= self.protection_barrier*self.underlying.loc[self.curr_date,0]):
              self.balance[self.client]  += self.nominal
              self.balance[self.seller] += (-self.nominal)
              self.finished=True
              print("the nominal : ", str(self.nominal), "has been reimbursed from seller to client. Product terminated")

            else:
              self.balance[self.client]  += self.nominal * ((self.underlying[self.curr_date()]/self.underlying[self.initial_date]) -1)
              self.balance[self.seller] += (-self.nominal * ((self.underlying[self.curr_date()]/self.underlying[self.initial_date]) -1))
              self.finished=True
              print("the remaining capital (- loss) : ", str(self.nominal * ((self.underlying[self.curr_date()]/self.underlying[self.initial_date]) -1)), "has been reimbursed from seller to client. Product terminated")

        else : self.balance[self.client]  += 0



In [ ]:
autocall_sim = Autocall_Athena_Sim("AC_test_Sim", 'CACIB', "2023-10-24")

autocall_sim.mint("Kathy", "ISIN1", 1, ["2022-01-03", "2022-04-01", "2022-07-01", "2022-10-03"], 1000, 0.1, 0.7, 'CBETHUSD', 4)


1000

In [ ]:
# test of the payoff function, we backtest as if autocall1 was created the 01/10/2021
autocall_sim.initial_date = "2021-01-10"

In [ ]:
for date in autocall_sim.underlying.index :
  autocall_sim.curr_date = date
  autocall_sim.event_coupon_autocall()


a coupon and the nominal :  1025.0 has been transferd from seller to client. Product terminated.


Another scenario

In [ ]:
autocall_sim2 = Autocall_Athena_Sim("AC_test_Sim2", 'CACIB', "2023-10-24")

autocall_sim2.mint("Kathy", "ISIN1", 1, ["2022-02-03", "2022-05-01", "2022-08-01", "2022-11-03"], 1000, 0.1, 0.9, 'CBETHUSD', 4)

autocall_sim2.initial_date = "2021-11-08"

In [ ]:
for date in autocall_sim2.underlying.index :
  autocall_sim2.curr_date = date
  autocall_sim2.event_coupon_autocall()


the nominal :  1000 has been reimbursed from seller to client. Product terminated
